### Загрузка библиотек

In [ ]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Функции для оптимизации

In [ ]:
def reduction_memory(df: pd.DataFrame):
    """function for reduction memory size"""
    
    df_c = df.copy()
    
    for column in df_c.columns:
    
        series = df_c[column]
        
        if series.dtype == 'float64':
            if (series != series.astype('float32')).sum() == 0:
                df_c[column] = series.astype('float32')
        elif series.dtype == 'int64':
            if (series != series.astype('int8')).sum() == 0:
                df_c[column] = series.astype('int8')
            elif (series != series.astype('int16')).sum() == 0:
                df_c[column] = series.astype('int16')
            elif (series != series.astype('int32')).sum() == 0:
                df_c[column] = series.astype('int32')
                
    return df_c

## 1. Прочитайте базу my_PJME_MW.csv и сделайте ее перрасчет (ресемплинг) в представление по 1 месяцу.

In [ ]:
df_pjme_mw = pd.read_csv('../input/lesson2-time-series/my_PJME_MW.csv', index_col=0, parse_dates=['Date'])
df_pjme_mw = reduction_memory(df_pjme_mw)
df_pjme_mw[:2]

In [ ]:
df_pjme_mw_month = df_pjme_mw.resample(rule='M').mean()
df_pjme_mw_month[:2]

## 2. Подберите параметры модели SARIMAX для этого ряда.

In [ ]:
#ACF
sm.graphics.tsa.plot_acf(df_pjme_mw_month['PJME_MW'].values, lags=40, title='ACF')
plt.show()
#PACF
sm.graphics.tsa.plot_pacf(df_pjme_mw_month['PJME_MW'].values, lags=40, title='PACF')
plt.show()

In [ ]:
mod_sarimax = sm.tsa.statespace.SARIMAX(df_pjme_mw_month['PJME_MW'], order=[0, 0, 1], 
                                        seasonal_order=[5, 2, 0, 12],
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)
result = mod_sarimax.fit()
print(result.summary().tables[0], result.summary().tables[1])

result.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
N = 90
plt.figure(figsize=(16,5))
plt.plot(df_pjme_mw_month['PJME_MW'][-N:], label='true')
plt.plot(result.predict()[-N:], label='pred')
plt.plot(result.forecast(), 'o', label='forecast')

plt.legend()
plt.show()

## 3. Сделайте вывод о том, как изменилась модель по сравнению с недельной.

In [ ]:
df_pjme_mw_week = df_pjme_mw.resample(rule='W').mean()
df_pjme_mw_week[:2]

In [ ]:
#ACF
sm.graphics.tsa.plot_acf(df_pjme_mw_week['PJME_MW'].values, lags=40, title='ACF')
plt.show()
#PACF
sm.graphics.tsa.plot_pacf(df_pjme_mw_week['PJME_MW'].values, lags=40, title='PACF')
plt.show()

In [ ]:
mod_sarimax = sm.tsa.statespace.SARIMAX(df_pjme_mw_week['PJME_MW'][-200:], order=[1, 0, 0], 
                                        seasonal_order=[3, 1, 0, 13],
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)
result = mod_sarimax.fit()
print(result.summary().tables[0], result.summary().tables[1])

result.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
N = 100
plt.figure(figsize=(16,5))
plt.plot(df_pjme_mw_week['PJME_MW'][-N:], label='true')
plt.plot(result.predict()[-N:], label='pred')
plt.plot(result.forecast(), 'o', label='forecast')

plt.legend(loc='lower right')
plt.show()

- ACF у месячной модели выглядиит более резко.
- Месячная модель строится гораздо быстрее, что позволяет больше внимания уделить настройкам.
- Из недельной модели приходилось брать выборку для ускорения обучения.
- Предсказанные значения у месячной модели выглядят точнее - вероятно из-за обобщенности

## 4. Постройте ACF, PACF и модель ARIMA для BRENT (lнедельный )

In [ ]:
df_brent = pd.read_csv('../input/ml-in-business/my_BRENT2019.csv', index_col=0, parse_dates=[0])
df_brent = reduction_memory(df_brent)
df_brent[:2]

In [ ]:
df_brent_week = df_brent.resample('W').mean()
df_brent_week[:2]

In [ ]:
# fill in empty value
get_oct = df_brent_week[(df_brent_week.index.year == 2011) & 
                        (df_brent_week.index.month == 10)]
df_brent_week.loc[df_brent_week['Значение'].isna(), 'Значение'] = get_oct['Значение'].mean()

In [ ]:
target = df_brent_week['Значение'].diff(1)[1:]

N = 40

#ACF
sm.graphics.tsa.plot_acf(target, lags=40, title='ACF')
plt.show()
#PACF
sm.graphics.tsa.plot_pacf(target.values[N:], lags=40, title='PACF')
plt.show()

In [ ]:
# ARIMA
model_arima = ARIMA(target, order=(1,0,0))
result = model_arima.fit(disp=False)

print(result.summary().tables[0], result.summary().tables[1])

y_p = result.predict(target.shape[0], target.shape[0], typ='levels')
y_f = result.forecast(target.shape[0])

plt.figure(figsize=(16,5))
plt.plot(target.index, target, label='true')
plt.plot(target.index, y_f[0], label='pred')
plt.plot(target.index, y_f[2], '--g')

plt.title('Arima')
plt.legend()
plt.show()

## 5. Оцените результат моделирования

ACF - показывает высокую кореляцию значений ряда<br>
ARIMA - показывает слишком обобщенный результат, из-за хаотичности данных

## 6. Сделайте предположение о моделях, которые лучше работают с техниками регрессии на основе AR I MA.

С ARIMA должны лучше работать стационарные модели, циклические, и трендовые - так как они омеют более-менее стабильную форму, которую модель может обобщить.<br>
Нестабильные модели обобщаются слишком сильно, что мешает ARIMA отследить зависимость в данных.